In [ ]:
# Scraping the minutes
from bs4 import BeautifulSoup
import requests
import re
import urllib.request
import os

# We are only interested in the "Transcripts" in this kernel
l_presconf = len("fomcpresconf")
presconf_links = {}

# Before storing the links, we have some pitfalls to notice:
# Since before 2014, all releases are stored in a different
# URL as that after 2014, we split the code into 2 subparts

for year in range(1982, 2020): # from 1982 - 2013
    if year < 2014:
        base_url = "https://www.federalreserve.gov/monetarypolicy/"
        path = "fomchistorical" + str(year) + ".htm"
        html_doc = requests.get(base_url + path)
        soup = BeautifulSoup(html_doc.content, 'html.parser')
        links = soup.find_all("a", string = re.compile("Transcript .*"))
        link_base_url = "https://www.federalreserve.gov"
        presconf_links[str(year)] = [link_base_url + link["href"] for link in links]
        print("Year Completed ", year)
    else:
        # After 2014, since all years are located in only one URL, we try to extract them out
        original_url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
        html_doc = requests.get(original_url)
        soup = BeautifulSoup(html_doc.content, 'html.parser')
        links = soup.find_all("a", string = re.compile("Press Conference"))
        link_base_url = "https://www.federalreserve.gov"
        # URL for each year
        base_url = []
        l_presconf = len("fomcpresconf")
        for link in links:
            p = re.compile("[^/]*$")
            if p.search(link.get("href")).group()[:(l_presconf+4)] == 'fomcpresconf' + str(year):
                base_url.append(link_base_url + link["href"])
        final_link = []
        for url in base_url:
            html = requests.get(url)
            soup = BeautifulSoup(html.content, 'html.parser')
            links = soup.find_all("a", string = re.compile("Press Conference Transcript"))
            link_base_url = "https://www.federalreserve.gov"
            for link in links:
                final_link.append(link_base_url + link["href"])
        presconf_links[str(year)] = final_link
        print("Year Completed ", year)

In [ ]:
# Now, scraping the press conference transcripts:
for year in presconf_links.keys():
    # if the folder does not exist, open it
    if not os.path.exists("./FOMCpctranscript/" + year):
        os.makedirs("./FOMCpctranscript/" + year)
    for link in presconf_links[year]:
        response = urllib.request.urlopen(str(link))
        name = re.search("[^/]*$", str(link))
        print(link)
        with open("./FOMCpctranscript/" + year + "/" + name.group(), 'wb') as f:
            f.write(response.read())
        print("file downloaded")